<h2>LU factorization with partial pivoting</h2>

This code factors a matrix A and solves Ax=b by forward and backward substitution.

In [1]:
# These are the standard imports for CS 111. 
# This list may change as the quarter goes on.

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import sparse
from scipy import linalg
import scipy.sparse.linalg as spla
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
%matplotlib tk

In [2]:
# Example of an upper triangular matrix

U = np.array([[2,7,1,8],[0,2,8,1],[0,0,8,2],[0,0,0,8]])
print(U)

[[2 7 1 8]
 [0 2 8 1]
 [0 0 8 2]
 [0 0 0 8]]


In [3]:
# Example of a unit lower triangular matrix

L = np.array([[1,0,0,0],[.5,1,0,0],[0,.5,1,0],[-.5,-.5,0,1]])
print(L)

[[ 1.   0.   0.   0. ]
 [ 0.5  1.   0.   0. ]
 [ 0.   0.5  1.   0. ]
 [-0.5 -0.5  0.   1. ]]


In [4]:
# Experiments with a random matrix

A = np.round(20*np.random.rand(5,5))
print(A)

[[18.  2. 17.  6.  5.]
 [16.  3. 10.  3.  5.]
 [ 9.  8. 17. 10.  3.]
 [11.  4.  4.  1. 11.]
 [ 5.  5. 15. 10.  4.]]


In [5]:
npla.matrix_rank(A)

5

In [6]:
npla.norm(A)

47.81213235152768

In [7]:
npla.cond(A)

85.47877059705628

In [8]:
# Creating a right-hand side for which we know the answer to Ax=b

xorig = np.round(10*np.random.rand(5))
print("original x:", xorig)
b = A @ xorig
print("right-hand side b:", b)

original x: [0. 2. 3. 2. 5.]
right-hand side b: [ 92.  67. 102.  77.  95.]


In [9]:
x = npla.solve(A,b)
print("computed x:", x)

computed x: [6.41462192e-16 2.00000000e+00 3.00000000e+00 2.00000000e+00
 5.00000000e+00]


In [10]:
error = x - xorig
print("error in x:", error)

error in x: [ 6.41462192e-16 -1.33226763e-15 -2.22044605e-15  4.44089210e-15
  0.00000000e+00]


In [11]:
print("relative norm of error:", npla.norm(x-xorig) / npla.norm(x))

relative norm of error: 7.99379432951357e-16


In [12]:
residual = b - A@x
print("residual:", residual)
print("relative norm of residual:", npla.norm(residual) / npla.norm(b))

residual: [0. 0. 0. 0. 0.]
relative norm of residual: 0.0


In [14]:
# Now a different matrix A and right-hand side vector b

A = np.array([[ 2. ,  7. ,  1. ,  8. ],
       [ 1. ,  5.5,  8.5,  5. ],
       [ 0. ,  1. , 12. ,  2.5],
       [-1. , -4.5, -4.5,  3.5]])
print("A:", A,'\n')
b = np.array([17. ,  2.5, -7. , 10.5])
print("b:", b)

A: [[ 2.   7.   1.   8. ]
 [ 1.   5.5  8.5  5. ]
 [ 0.   1.  12.   2.5]
 [-1.  -4.5 -4.5  3.5]] 

b: [17.   2.5 -7.  10.5]


In [15]:
# We used Gaussian elimination on the blackboard to triangularize A, giving U

print(U)

[[2 7 1 8]
 [0 2 8 1]
 [0 0 8 2]
 [0 0 0 8]]


In [16]:
# During Gaussian elimination, we wrote down the multipliers in 
# a lower triangular array and then put ones on the diagonal, giving L

print(L)

[[ 1.   0.   0.   0. ]
 [ 0.5  1.   0.   0. ]
 [ 0.   0.5  1.   0. ]
 [-0.5 -0.5  0.   1. ]]


In [17]:
# The theorem: Gaussian elimination factors A as the product L time U

print( L @ U)
print()
print(A)

[[ 2.   7.   1.   8. ]
 [ 1.   5.5  8.5  5. ]
 [ 0.   1.  12.   2.5]
 [-1.  -4.5 -4.5  3.5]]

[[ 2.   7.   1.   8. ]
 [ 1.   5.5  8.5  5. ]
 [ 0.   1.  12.   2.5]
 [-1.  -4.5 -4.5  3.5]]


In [18]:
def LUfactorNoPiv(A):
    """Factor a square matrix, A == L @ U (no partial pivoting)
    Parameters: 
      A: the matrix.
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
            
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    return (L, U)

In [19]:
def Lsolve(L, b):
    """Forward solve a unit lower triangular system Ly = b for y
    Parameters: 
      L: the matrix, must be square, lower triangular, with ones on the diagonal
      b: the right-hand side vector
    Output:
      y: the solution vector to L @ y == b
    """
    
    # Check the input
    m, n = L.shape
    assert m == n, "matrix L must be square"
    assert np.all(np.tril(L) == L), "matrix L must be lower triangular"
    assert np.all(np.diag(L) == 1), "matrix L must have ones on the diagonal"
    
    # Make a copy of b that we will transform into the solution
    y = b.astype(np.float64).copy()
    
    # Forward solve
    for col in range(n):
        y[col+1:] -= y[col] * L[col+1:, col]
        
    return y

In [35]:
def Usolve(U, y):
    """Backward solve an upper triangular system Ux = y for x
    Parameters: 
      U: the matrix, must be square, upper triangular, with nonzeros on the diagonal
      y: the right-hand side vector
    Output:
      x: the solution vector to U @ x == y
    """
    
    print("you will write Usolve in hw2")
        
    return 

In [21]:
L,U = LUfactorNoPiv(A)
y = Lsolve(L,b)
print("y:", y)
x = Usolve(U,y)
print("\nx:", x)
print("\nresidual norm:", npla.norm(b - A @ x))

y: [17. -6. -4. 16.]

x: [ 1.  0. -1.  2.]

residual norm: 0.0


In [22]:
A @ [1,0,-1,2]

array([17. ,  2.5, -7. , 10.5])

In [23]:
# But LU factorization (without pivoting) fails if it encounters a zero pivot

A = np.array([[0, 1], [1, 2]])
L,U = LUfactorNoPiv(A)


AssertionError: pivot is zero, can't continue

In [25]:
def LUfactor(A, pivoting = True):
    """Factor a square matrix with partial pivoting, A[p,:] == L @ U
    Parameters: 
      A: the matrix.
      pivoting: whether or not to do partial pivoting
    Outputs (in order):
      L: the lower triangular factor, same dimensions as A, with ones on the diagonal
      U: the upper triangular factor, same dimensions as A
      p: the permutation vector that permutes the rows of A by partial pivoting
    """
    
    # Check the input
    m, n = A.shape
    assert m == n, 'input matrix A must be square'
    
    # Initialize p to be the identity permutation
    p = np.array(range(n))
    
    # Make a copy of the matrix that we will transform into L and U
    LU = A.astype(np.float64).copy()
    
    # Eliminate each column in turn
    for piv_col in range(n):
        
        # Choose the pivot row and swap it into place
        if pivoting:
            piv_row = piv_col + np.argmax(LU[piv_col:, piv_col]) 
            assert LU[piv_row, piv_col] != 0., "can't find nonzero pivot, matrix is singular"
            LU[[piv_col, piv_row], :]  = LU[[piv_row, piv_col], :]
            p[[piv_col, piv_row]]      = p[[piv_row, piv_col]]
            
        # Update the rest of the matrix
        pivot = LU[piv_col, piv_col]
        assert pivot != 0., "pivot is zero, can't continue"
        for row in range(piv_col + 1, n):
            multiplier = LU[row, piv_col] / pivot
            LU[row, piv_col] = multiplier
            LU[row, (piv_col+1):] -= multiplier * LU[piv_col, (piv_col+1):]
            
    # Separate L and U in the result
    U = np.triu(LU)
    L = LU - U + np.eye(n)
    
    return (L, U, p)

In [30]:
# Our 2-by-2 counterexample again

print('A:\n', A)
L, U, p = LUfactor(A)
print('\nL:\n', L)
print('\nU:\n', U)
print('\np: ', p)

A:
 [[19.  0. 16. 19. 18.]
 [ 9. 10. 12. 16. 12.]
 [ 4. 11.  1. 20. 14.]
 [ 5. 15. 15. 20.  7.]
 [13.  7. 16. 14.  1.]]

L:
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.63157895e-01  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 6.84210526e-01  4.66666667e-01  1.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 4.73684211e-01  6.66666667e-01 -1.58000000e+02  1.00000000e+00
   0.00000000e+00]
 [ 2.10526316e-01  7.33333333e-01 -5.86000000e+02  3.69190326e+00
   1.00000000e+00]]

U:
 [[ 1.90000000e+01  0.00000000e+00  1.60000000e+01  1.90000000e+01
   1.80000000e+01]
 [ 0.00000000e+00  1.50000000e+01  1.07894737e+01  1.50000000e+01
   2.26315789e+00]
 [ 0.00000000e+00  0.00000000e+00  1.75438596e-02 -6.00000000e+00
  -1.23719298e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -9.51000000e+02
  -1.95280000e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -3.18513144e+01]]

p:  [0 3 4 1 2]


In [31]:
# A larger example of LU with partial pivoting

A = np.round(20*np.random.rand(5,5))
print('matrix A:\n', A)
xorig = np.round(10*np.random.rand(5))
print('\noriginal x:', xorig)
b = A @ xorig
print('\nright-hand side b:', b)

matrix A:
 [[ 0. 11. 15. 16.  5.]
 [ 9.  8. 10. 15. 14.]
 [18. 13. 13.  7. 12.]
 [10. 12.  8.  1. 14.]
 [ 3.  5.  2. 14. 10.]]

original x: [7. 2. 6. 2. 8.]

right-hand side b: [184. 281. 340. 256. 151.]


In [33]:
# Factor the larger example

L, U, p = LUfactor(A)
print("norm of difference between L times U and permuted A:", npla.norm( L@U - A[p,:]))

norm of difference between L times U and permuted A: 7.136584070361083e-15


In [34]:
# Solve with the larger example

y = Lsolve(L,b[p])
print("y:", y)
x = Usolve(U,y)
print("\nx:", x)
print("\nresidual norm:", npla.norm(b - A @ x))

y: [340.         184.          85.90909091 284.97916667 103.91794872]

x: [7. 2. 6. 2. 8.]

residual norm: 6.355287432313019e-14
